In [1]:
%load_ext autoreload

In [25]:
%autoreload

import audl_advanced_stats as audl
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

In [4]:
%%time
s = audl.Season()
# s.get_game_qc()

Wall time: 1 ms


In [367]:
g = audl.Game(s.get_game_info()["url"].iloc[0])
# g.get_home_events(upload=True)
# g.get_away_events(upload=True)
fig = g.visual_game_flow(color="o_point", home=True, qc=True)
fig

Segments w/o 7 players: 0


In [434]:
g.get_home_roster().sort_values("id")

,id,team_season_id,player_id,jersey_number,first_name,last_name,ext_player_id,active
15,9448,210,1543,2,Ryan,Osgar,rosgar,True
16,9449,210,1545,1,Ryan,Saffa,rsaffa,False
20,9450,210,2093,3,Elliott,Chartock,echartock,True
0,9451,210,174,4,Ben,Jagt,bjagt,True
18,9452,210,2001,8,TJ,Stanton,tstanton,True
24,9453,210,2637,9,Solomon,Rueschemeyer-Bailey,srueschem,True
3,9454,210,408,10,Conor,Kline,ckline,True
4,9455,210,747,11,Jack,Williams,jwilliams,True
9,9456,210,1207,12,Matt,LeMar,mlemar,True
6,9457,210,874,14,Joe,Ouellette,jouellett,True


In [6]:
g = audl.Game(s.get_game_info()["url"].iloc[0])
g.get_player_points_played(df=g.get_home_events(), start_only=True)

,playerid,total_points,o_points,o_point_scores,o_point_score_pct,o_point_noturns,o_point_noturn_pct,d_points,d_point_scores,d_point_score_pct,d_point_turns,d_point_turn_pct
0,9448,20,20,15,0.750000,13,0.650000,0,0,NaN,0,NaN
1,9450,17,17,13,0.764706,10,0.588235,0,0,NaN,0,NaN
2,9451,22,20,15,0.750000,13,0.650000,2,0,0.000000,0,0.000000
3,9452,5,0,0,NaN,0,NaN,5,1,0.200000,1,0.200000
4,9453,4,0,0,NaN,0,NaN,4,1,0.250000,1,0.250000
5,9454,17,17,13,0.764706,10,0.588235,0,0,NaN,0,NaN
6,9455,21,20,15,0.750000,13,0.650000,1,0,0.000000,0,0.000000
7,9456,14,0,0,NaN,0,NaN,14,3,0.214286,4,0.285714
8,9457,4,0,0,NaN,0,NaN,4,1,0.250000,1,0.250000
9,9458,16,0,0,NaN,0,NaN,16,3,0.187500,4,0.250000


In [5]:
g = audl.Game(s.get_game_info()["url"].iloc[0])
g.get_player_yards(df=g.get_home_events())

,playerid,xyards_throwing,xyards_throwing_center,yyards_throwing,yyards_throwing_center,yyards_raw_throwing,yyards_raw_throwing_center,yards_throwing,yards_throwing_center,yards_raw_throwing,...,xyards_receiving,xyards_receiving_center,yyards_receiving,yyards_receiving_center,yyards_raw_receiving,yyards_raw_receiving_center,yards_receiving,yards_receiving_center,yards_raw_receiving,yards_raw_receiving_center
0,9448,348.33,0.00,385.01,0.00,484.31,0.00,606.081896,0.000000,698.543886,...,386.12,0.00,178.36,0.00,194.93,0.00,519.324664,0.000000,535.700945,0.000000
1,9450,575.44,41.16,415.42,15.74,443.61,15.74,822.880281,44.511069,849.495800,...,487.01,85.62,1.94,34.59,12.03,34.59,617.031984,94.379218,626.509713,94.379218
2,9451,280.76,0.00,23.80,0.00,57.50,0.00,385.570999,0.000000,411.095938,...,373.67,0.00,403.76,0.00,424.96,0.00,629.985017,0.000000,648.812143,0.000000
3,9453,29.87,0.00,43.50,0.00,57.60,0.00,56.753430,0.000000,70.804856,...,54.72,0.00,-16.38,0.00,-16.38,0.00,60.642239,0.000000,60.642239,0.000000
4,9454,86.28,0.00,-15.95,0.00,-15.95,0.00,97.078674,0.000000,97.078674,...,68.78,0.00,70.43,0.00,106.02,0.00,113.087502,0.000000,142.923277,0.000000
5,9455,362.51,0.00,163.94,0.00,185.14,0.00,514.801617,0.000000,533.968240,...,431.12,0.00,221.12,0.00,239.41,0.00,543.632028,0.000000,559.662733,0.000000
6,9456,126.13,0.00,-2.19,0.00,-2.19,0.00,134.650412,0.000000,134.650412,...,41.17,0.00,22.57,0.00,22.57,0.00,57.457237,0.000000,57.457237,0.000000
7,9457,2.84,0.00,19.51,0.00,19.51,0.00,19.715621,0.000000,19.715621,...,10.49,0.00,14.49,0.00,14.49,0.00,17.888549,0.000000,17.888549,0.000000
8,9458,45.92,0.00,-13.41,0.00,-13.41,0.00,49.028935,0.000000,49.028935,...,44.96,0.00,30.87,0.00,30.87,0.00,68.086738,0.000000,68.086738,0.000000
9,9459,29.80,0.00,1.53,0.00,1.53,0.00,29.839251,0.000000,29.839251,...,19.89,0.00,4.15,0.00,4.15,0.00,20.318332,0.000000,20.318332,0.000000


In [26]:
g = audl.Game(s.get_game_info()["url"].iloc[0])
g.get_player_touches(df=g.get_home_events())

,playerid,throwaways,completions,receptions,drops,stalls,assists,goals,blocks,callahans,turnovers,plus_minus,throw_attempts,catch_attempts,completion_pct,reception_pct
0,9448,1.0,26.0,28,0.0,0.0,8.0,1.0,0.0,0.0,1.0,8.0,27.0,28.0,0.962963,1.000000
1,9450,3.0,46.0,40,0.0,0.0,2.0,1.0,0.0,0.0,3.0,0.0,49.0,40.0,0.938776,1.000000
2,9451,2.0,22.0,26,0.0,0.0,3.0,2.0,0.0,0.0,2.0,3.0,24.0,26.0,0.916667,1.000000
3,9463,1.0,19.0,15,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,20.0,15.0,0.950000,1.000000
4,9465,1.0,26.0,19,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,27.0,19.0,0.962963,1.000000
5,9469,1.0,2.0,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,3.0,3.0,0.666667,1.000000
6,9453,0.0,3.0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.000000,1.000000
7,9454,0.0,4.0,7,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,4.0,7.0,1.000000,1.000000
8,9455,0.0,24.0,25,0.0,0.0,2.0,2.0,0.0,0.0,0.0,4.0,24.0,25.0,1.000000,1.000000
9,9456,0.0,6.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,1.000000,1.000000


In [366]:
g = audl.Game(s.get_game_info()["url"].iloc[0])
g.get_home_events().groupby(['9466', 'o_point'])["point_number"].nunique()

9466  o_point
0     False      19
      True        3
1     False       2
      True       20
Name: point_number, dtype: int64

In [438]:
df = g.get_home_events()
start_only = True

points_played = []
# Get all player IDs for the team in this game
players = [x for x in list(df) if x.isdigit()]
for playerid in players:
    if start_only:
        dftemp = df.groupby(["point_number",]).head(1)
    else:
        dftemp = df

    # Get the number of o and d points each player played
    player_points = (
        dftemp.assign(turnover = lambda x: x["num_turnovers"] > 0)
        .groupby([playerid, "o_point", "turnover", "point_outcome"])["point_number"]
        .nunique()
        .rename("points")[1]
        .reset_index()
        .assign(playerid=playerid)
    )
    points_played.append(player_points)

# Combine all players into a single dataframe
dfout = (
    pd.concat(points_played, ignore_index=True)
    .assign(
        o_point=lambda x: x["o_point"].map({True: "o_point", False: "d_point"}),
        turnover=lambda x: x["turnover"].map({True: "at least 1 turn", False: "no turns"}),
    )
    .set_index(["playerid", "o_point", "turnover", "point_outcome"])
    .unstack(level=["o_point", "turnover", "point_outcome"], fill_value=0)
    .reset_index()
)
dfout.columns = ["_".join(col).rstrip("_") for col in dfout.columns.values]
o_point_cols = [x for x in list(dfout) if "o_point" in x]
o_point_score_cols = [
    x
    for x in list(dfout)
    if ("o_point" in x)
    and (("Score" in x) or ("Callahan" in x))
    and ("Opponent" not in x)
]
o_point_noturn_cols = [
    x
    for x in list(dfout)
    if ("o_point" in x)
    and ("no turns" in x)
]
d_point_cols = [x for x in list(dfout) if "d_point" in x]
d_point_score_cols = [
    x
    for x in list(dfout)
    if ("d_point" in x)
    and (("Score" in x) or ("Callahan" in x))
    and ("Opponent" not in x)
]
d_point_turn_cols = [
    x
    for x in list(dfout)
    if ("d_point" in x)
    and ("at least 1 turn" in x)
]
dfout["total_points"] = dfout.sum(axis=1)
dfout["o_points"] = dfout[o_point_cols].sum(axis=1)
dfout["d_points"] = dfout[d_point_cols].sum(axis=1)
dfout["o_point_scores"] = dfout[o_point_score_cols].sum(axis=1)
dfout["d_point_scores"] = dfout[d_point_score_cols].sum(axis=1)
dfout["o_point_noturns"] = dfout[o_point_noturn_cols].sum(axis=1)
dfout["d_point_turns"] = dfout[d_point_turn_cols].sum(axis=1)
dfout["o_point_score_pct"] = dfout["o_point_scores"] / dfout["o_points"]
dfout["d_point_score_pct"] = dfout["d_point_scores"] / dfout["d_points"]
dfout["o_point_noturn_pct"] = dfout["o_point_noturns"] / dfout["o_points"]
dfout["d_point_turn_pct"] = dfout["d_point_turns"] / dfout["d_points"]

dfout

,playerid,points_o_point_no turns_End of Period,points_o_point_no turns_Score,points_o_point_at least 1 turn_Opponent Score,points_o_point_at least 1 turn_Score,points_d_point_no turns_End of Period,points_d_point_no turns_Opponent Score,points_d_point_at least 1 turn_Score,points_d_point_at least 1 turn_Opponent Score,total_points,o_points,d_points,o_point_scores,d_point_scores,o_point_noturns,d_point_turns,o_point_score_pct,d_point_score_pct,o_point_noturn_pct,d_point_turn_pct
0,9448,1,12,4,3,0,0,0,0,20,20,0,15,0,13,0,0.750000,NaN,0.650000,NaN
1,9450,0,10,4,3,0,0,0,0,17,17,0,13,0,10,0,0.764706,NaN,0.588235,NaN
2,9451,1,12,4,3,1,1,0,0,22,20,2,15,0,13,0,0.750000,0.000000,0.650000,0.000000
3,9452,0,0,0,0,0,4,1,0,5,0,5,0,1,0,1,NaN,0.200000,NaN,0.200000
4,9453,0,0,0,0,0,3,1,0,4,0,4,0,1,0,1,NaN,0.250000,NaN,0.250000
5,9454,0,10,4,3,0,0,0,0,17,17,0,13,0,10,0,0.764706,NaN,0.588235,NaN
6,9455,1,12,4,3,1,0,0,0,21,20,1,15,0,13,0,0.750000,0.000000,0.650000,0.000000
7,9456,0,0,0,0,0,10,3,1,14,0,14,0,3,0,4,NaN,0.214286,NaN,0.285714
8,9457,0,0,0,0,0,3,1,0,4,0,4,0,1,0,1,NaN,0.250000,NaN,0.250000
9,9458,0,0,0,0,1,11,3,1,16,0,16,0,3,0,4,NaN,0.187500,NaN,0.250000


In [387]:
g.get_home_events().groupby(['point_number',]).head(1).groupby([playerid, 'o_point'])["point_number"].nunique().rename("points_played")[1].reset_index().assign(playerid=playerid)

,o_point,points_played,playerid
0,False,15,9550
